In [2]:
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console
from tools import web_search_tool, save_report_to_md


In [3]:
model_client = OpenAIChatCompletionClient(
    model="gpt-5-nano-2025-08-07",
)


In [4]:
research_planner = AssistantAgent(
    "research_planner",
    description="A strategic research coordinator that breaks down complex questions into research subtasks",
    model_client=model_client,
    system_message="""You are a research planning specialist. Your job is to create a focused research plan.

For each research question, create a FOCUSED research plan with:

1. **Core Topics**: 2-3 main areas to investigate
2. **Search Queries**: Create 3-5 specific search queries covering:
   - Latest developments and news
   - Key statistics or data
   - Expert analysis or studies
   - Future outlook

Keep the plan focused and achievable. Quality over quantity.""",
)

research_agent = AssistantAgent(
    "research_agent",
    description="A web research specialist that searches and extracts content",
    tools=[web_search_tool],
    model_client=model_client,
    system_message="""You are a web research specialist. Your job is to conduct focused searches based on the research plan.

RESEARCH STRATEGY:
1. **Execute 3-5 searches** from the research plan
2. **Extract key information** from the results:
   - Main facts and statistics
   - Recent developments
   - Expert opinions
   - Important context

3. **Quality focus**:
   - Prioritize authoritative sources
   - Look for recent information (within 2 years)
   - Note diverse perspectives

After completing the searches from the plan, summarize what you found. Your goal is to gather 5-10 quality sources.""",
)

research_analyst = AssistantAgent(
    "research_analyst",
    description="An expert analyst that creates research reports",
    model_client=model_client,
    system_message="""You are a research analyst. Create a comprehensive report from the gathered research.

CREATE A RESEARCH REPORT with:

## Executive Summary
- Key findings and conclusions
- Main insights

## Background & Current State
- Current landscape
- Recent developments
- Key statistics and data

## Analysis & Insights
- Main trends
- Different perspectives
- Expert opinions

## Future Outlook
- Emerging trends
- Predictions
- Implications

## Sources
- List all sources used

Write a clear, well-structured report based on the research gathered. End with "REPORT_COMPLETE" when finished.""",
)

quality_reviewer = AssistantAgent(
    "quality_reviewer",
    description="A quality assurance specialist that evaluates research completeness and accuracy",
    tools=[save_report_to_md],
    model_client=model_client,
    system_message="""You are a quality reviewer. Your job is to check if the research analyst has produced a complete research report.

Look for:
- A comprehensive research report from the research analyst that ends with "REPORT_COMPLETE"
- The research question is fully answered
- Sources are cited and reliable
- The report includes summary, key information, analysis, and sources

When you see a complete research report that ends with "REPORT_COMPLETE":
1. First, use the save_report_to_md tool to save the report to report.md
2. Then say: "The research is complete. The report has been saved to report.md. Please review the report and let me know if you approve it or need additional research."

If the research analyst has NOT yet created a complete report, tell them to create one now.""",
)

research_enhancer = AssistantAgent(
    "research_enhancer",
    description="A specialist that identifies critical gaps only",
    model_client=model_client,
    system_message="""You are a research enhancement specialist. Your job is to identify ONLY CRITICAL gaps.

Review the research and ONLY suggest additional searches if there are MAJOR gaps like:
- Completely missing recent developments (last 6 months)
- No statistics or data at all
- Missing a crucial perspective that was specifically asked for

If the research covers the basics reasonably well, say: "The research is sufficient to proceed with the report."

Only suggest 1-2 additional searches if absolutely necessary. We prioritize getting a good report done rather than perfect coverage.""",
)

user_proxy = UserProxyAgent(
    "user_proxy",
    description="Human reviewer who can request additional research or approve final results",
    input_func=input,
)


In [6]:
selector_prompt = """
Choose the best agent for the current task based on the conversation history:

{roles}

Current conversation:
{history}

Available agents:
- research_planner: Plan the research approach (ONLY at the start)
- research_agent: Search for and extract content from web sources (after planning)
- research_enhancer: Identify CRITICAL gaps only (use sparingly)
- research_analyst: Write the final research report
- quality_reviewer: Check if a complete report exists
- user_proxy: Ask the human for feedback

WORKFLOW:
1. If no planning done yet → select research_planner
2. If planning done but no research → select research_agent  
3. After research_agent completes initial searches → select research_enhancer ONCE
4. If enhancer says "sufficient to proceed" → select research_analyst
5. If enhancer suggests critical searches → select research_agent ONCE more then research_analyst
6. If research_analyst said "REPORT_COMPLETE" → select quality_reviewer
7. If quality_reviewer asked for user feedback → select user_proxy

IMPORTANT: After research_agent has searched 2 times maximum, proceed to research_analyst regardless.

Pick the agent that should work next based on this workflow."""


In [8]:
text_termination = TextMentionTermination("APPROVED")
max_message_termination = MaxMessageTermination(max_messages=50)
termination_condition = text_termination | max_message_termination

team = SelectorGroupChat(
    participants=[
        research_agent,
        research_analyst,
        research_enhancer,
        research_planner,
        quality_reviewer,
        user_proxy,
    ],
    selector_prompt=selector_prompt,
    model_client=model_client,
    allow_repeated_speaker=True,
    termination_condition=termination_condition,
)


In [9]:
await Console(
    team.run_stream(task="바이브 코딩과, 소프트웨어 개발자의 미래에 대해서 조사해"),
)


---------- TextMessage (user) ----------
바이브 코딩과, 소프트웨어 개발자의 미래에 대해서 조사해
---------- TextMessage (research_planner) ----------
좋습니다. 다만 “바이브 코딩”이 어떤 개념을 가리키는지 정확히 파악해야 계획을 정확하게 맞출 수 있습니다.

확인 질문
1) 바이브 코딩이 특정 기술/도구/프레임워크의 이름인가요, 아니면 일반적인 개념으로서의 분위기(바이브) 기반 코딩이나 감정/환경 인식과 연계된 코딩 방식인가요?
2) 연구의 주된 초점은 학술 연구(논문 중심), 업계 트렌드(실무 영향), 또는 교육/직무 전망 중 어느 쪽에 맞추길 원하나요?
3) 산출물 형식은 요약 보고서, 비교 분석 표, 권고안 등의 어떤 형식을 선호하시나요?
4) 최근 2–3년의 최신 내용까지 포함하는 것이 좋나요, 아니면 더 넓은 시계열을 다루어도 되나요?

확인되면, 위의 방향에 맞춰 바로 FOCUSED 연구 계획을 드리겠습니다. 아래는 한 가지 가정 하에 빠르게 예시 구조를 보여드리는 형태이며, 실제 해석이 확정되면 그에 맞춰 세부를 조정하겠습니다.

예시(가정: 바이브 코딩을 분위기/감정 기반 코딩이나 인터랙티브 코딩이라는 일반 개념으로 해석)
- Core Topics (2–3개)
  1) 바이브 코딩의 정의와 현황: 용어 정의, 현재 사용 사례, 도구/플랫폼 현황
  2) 실용성 및 도입 사례: 산업별 적용 사례, 장점·한계, 도입 장애 요인
  3) 소프트웨어 개발자의 미래: AI 보조 coding의 진화, 스킬 체인 변화, 고용 시장과 교육의 적응

- Search Queries (각 코어 토픽당 3–5개 예시)
  1) 최신 개발 동향/뉴스
     - "바이브 코딩 최신 동향" / "vibe coding latest developments"
     - "감정 기반 코딩 도구 신제품" / "emotion-based coding tools 2024-2025"
  2) 주요 통계나 데이

TaskResult(messages=[TextMessage(id='a8b57596-d866-46ae-b3d8-2506d6558437', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 8, 12, 31, 38, 958442, tzinfo=datetime.timezone.utc), content='바이브 코딩과, 소프트웨어 개발자의 미래에 대해서 조사해', type='TextMessage'), TextMessage(id='8140f2b7-c2dd-4f4d-b1d1-cd147763a6f9', source='research_planner', models_usage=RequestUsage(prompt_tokens=129, completion_tokens=4405), metadata={}, created_at=datetime.datetime(2025, 9, 8, 12, 32, 13, 52811, tzinfo=datetime.timezone.utc), content='좋습니다. 다만 “바이브 코딩”이 어떤 개념을 가리키는지 정확히 파악해야 계획을 정확하게 맞출 수 있습니다.\n\n확인 질문\n1) 바이브 코딩이 특정 기술/도구/프레임워크의 이름인가요, 아니면 일반적인 개념으로서의 분위기(바이브) 기반 코딩이나 감정/환경 인식과 연계된 코딩 방식인가요?\n2) 연구의 주된 초점은 학술 연구(논문 중심), 업계 트렌드(실무 영향), 또는 교육/직무 전망 중 어느 쪽에 맞추길 원하나요?\n3) 산출물 형식은 요약 보고서, 비교 분석 표, 권고안 등의 어떤 형식을 선호하시나요?\n4) 최근 2–3년의 최신 내용까지 포함하는 것이 좋나요, 아니면 더 넓은 시계열을 다루어도 되나요?\n\n확인되면, 위의 방향에 맞춰 바로 FOCUSED 연구 계획을 드리겠습니다. 아래는 한 가지 가정 하에 빠르게 예시 구조를 보여드리는 형태이며, 실제 해석이 확정되면 그에 맞춰 세부를 조정하겠습니